In [1]:
import os
import copy
import numpy as np
import pandas as pd
from osgeo import gdal
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
plt.rcParams['font.sans-serif']='Times New Roman'

In [2]:
def read_image(file):
    '''
    读取栅格图像
    file: 文件名
    '''
    img = gdal.Open(file)
    try:
        data = img.ReadAsArray(0, 0, img.RasterXSize, img.RasterYSize)
    except:
        pass        
    return data

def make_raster(in_ds,fn,data,data_type,nodata=None):
    """
    Create a one-band GeoTIFF.
    in_ds      -datasource to copy projection and geotransform from
    fn         -path to the file to create
    data       -Numpy array containing data to write
    data_type  -output data type
    nodata     -optional Nodata value
    """
    driver = gdal.GetDriverByName('GTiff')
    in_ds = gdal.Open(in_ds)
    out_ds = driver.Create(fn,in_ds.RasterXSize,in_ds.RasterYSize,1,data_type)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())
    out_band = out_ds.GetRasterBand(1)
    if nodata is not None:
        out_band.SetNoDataValue(nodata)
    out_band.WriteArray(data)
    out_band.FlushCache()
    out_band.ComputeStatistics(False)
    
def Raster_reprojcect_resample_mask(out_file,in_file,mask_shape):
    img = gdal.Open(in_file)
    GeoTransform = img.GetGeoTransform()
    out_srs = img.GetProjection()
    ds = gdal.Warp(out_file, 
                    in_file, 
                    dstSRS=out_srs,
                    cutlineDSName = mask_shape,
                    cropToCutline = True,
                    copyMetadata=True,
                    outputType=gdal.GDT_Float32, 
                    xRes=GeoTransform[1], 
                    yRes=GeoTransform[1])
    ds = None
    
def Extract_value_from_raster(raster_path, condinate_x, condinate_y):
    '''
    raster_path 栅格图像路径
    condinate_x 
    condinate_y 
    '''
    img = gdal.Open(raster_path)
    rows = img.RasterYSize
    cols = img.RasterXSize
    bands = img.RasterCount
    
    GeoTransform = img.GetGeoTransform()
    
    values = []
    for i in range(len(condinate_x)):
        
        x = condinate_x.iloc[i]
        y = condinate_y.iloc[i]
        
        xOffset = int((x-GeoTransform[0])/GeoTransform[1])
        yOffset = int((y-GeoTransform[3])/GeoTransform[5])

        band = img.GetRasterBand(1)
        values = np.append(values,band.ReadAsArray(xOffset, yOffset,1,1))
    return values

In [3]:
# --------------数据筛选-----------------
Path = '/Users/zhangperry/Desktop/TYPE.CSV'
Forest_data = pd.read_csv(Path,header=0,index_col=0)
# ---------------特征提取-------------------
Forest_data['AK'] = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/AK.tif',     Forest_data['lon'], Forest_data['lat'])
Forest_data['Altitude']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/altitude.tif',      Forest_data['lon'], Forest_data['lat'])
Forest_data['AN']    = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/AN.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['AP']   = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/AP.tif',   Forest_data['lon'], Forest_data['lat'])
Forest_data['Aspect']   = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/aspect.tif',   Forest_data['lon'], Forest_data['lat'])
Forest_data['Awch']   = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/awch.tif',   Forest_data['lon'], Forest_data['lat'])
Forest_data['AWCts']   = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/AWCts.tif',   Forest_data['lon'], Forest_data['lat'])
Forest_data['BDRLOG']   = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/BDRLOG.tif',   Forest_data['lon'], Forest_data['lat'])
Forest_data['BDTICM']   = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/BDTICM.tif',   Forest_data['lon'], Forest_data['lat'])
Forest_data['BLDFIE']   = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/BLDFIE.tif',   Forest_data['lon'], Forest_data['lat'])
Forest_data['CECSOL']   = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/CECSOL.tif',   Forest_data['lon'], Forest_data['lat'])
Forest_data['CLYPPT']   = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/CLYPPT.tif',   Forest_data['lon'], Forest_data['lat'])
Forest_data['DIF']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/DIF.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['LOS']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/LOS.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['OCDENS']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/OCDENS.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['OCSTHA']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/OCSTHA.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['ORCDRC']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/ORCDRC.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['P_pet']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/P_pet.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['P_pre']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/P_pre.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['P_tmp']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/P_tmp.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['PHICKL']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/PHICKL.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['PHIHOX']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/PHIHOX.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['Slope']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/slope.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['SLTPPT']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/SLTPPT.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['SNDPPT']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/SNDPPT.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data['WWP']  = Extract_value_from_raster('/Users/zhangperry/Desktop/11_15_22/WWP.tif',  Forest_data['lon'], Forest_data['lat'])
Forest_data = Forest_data.dropna()
Forest_data = Forest_data.reset_index()
Forest_data.drop(Forest_data.index[np.where(Forest_data['Aspect']<-9999)], inplace=True)

In [ ]:
X = Forest_data.iloc[:,4:].values
y = Forest_data.iloc[:, 3].values.astype('str')

# --------------------------树种分布建模------------------------------
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)
Classifier = RandomForestClassifier(n_estimators=1300,oob_score=True)
Classifier.fit(X_train, y_train)
y_pred = Classifier.predict(X_test)
y_pred_proba = Classifier.predict_proba(X_test)

# ------------预测准确率--------------------
print(metrics.classification_report(y_test, y_pred))
print(metrics.cohen_kappa_score(y_test, y_pred))

# -------------ROC曲线----------------------
Species_list = np.unique(y)
Species_list 
for i in np.arange(len(Species_list)):
    
    ytest = copy.deepcopy(y_test)

    ytest[np.where(ytest != Species_list[i])] = 0
    ytest[np.where(ytest == Species_list[i])] = 1

    y_pred_proba = Classifier.predict_proba(X_test)[:,i]
    fpr, tpr, threshold = metrics.roc_curve(list(np.int32(ytest)), list(y_pred_proba))
    roc_auc = metrics.auc(fpr, tpr)
    print(roc_auc)
    plt.plot(fpr, tpr)
    # plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive rate', size = 18)
plt.xlabel('False Positive rate', size = 18)
plt.show()

# ----------------树种分布概率预测-------------------
# -------------------图像特征------------------------
AK = read_image('/Users/zhangperry/Desktop/11_15_22/AK.tif').flatten()
Altitude  = read_image('/Users/zhangperry/Desktop/11_15_22/altitude.tif').flatten()
AN    = read_image('/Users/zhangperry/Desktop/11_15_22/AN.tif').flatten()
AP   = read_image('/Users/zhangperry/Desktop/11_15_22/AP.tif').flatten()
Aspect   = read_image('/Users/zhangperry/Desktop/11_15_22/aspect.tif').flatten()
Awch   = read_image('/Users/zhangperry/Desktop/11_15_22/awch.tif').flatten()
AWCts   = read_image('/Users/zhangperry/Desktop/11_15_22/AWCts.tif').flatten()
BDRLOG   = read_image('/Users/zhangperry/Desktop/11_15_22/BDRLOG.tif').flatten()
BDTICM   = read_image('/Users/zhangperry/Desktop/11_15_22/BDTICM.tif').flatten()
BLDFIE   = read_image('/Users/zhangperry/Desktop/11_15_22/BLDFIE.tif').flatten()
CECSOL   = read_image('/Users/zhangperry/Desktop/11_15_22/CECSOL.tif').flatten()
CLYPPT   = read_image('/Users/zhangperry/Desktop/11_15_22/CLYPPT.tif').flatten()
DIF  = read_image('/Users/zhangperry/Desktop/11_15_22/DIF.tif').flatten()
LOS  = read_image('/Users/zhangperry/Desktop/11_15_22/LOS.tif').flatten()
OCDENS  = read_image('/Users/zhangperry/Desktop/11_15_22/OCDENS.tif').flatten()
OCSTHA  = read_image('/Users/zhangperry/Desktop/11_15_22/OCSTHA.tif').flatten()
ORCDRC  = read_image('/Users/zhangperry/Desktop/11_15_22/ORCDRC.tif').flatten()
P_pet  = read_image('/Users/zhangperry/Desktop/11_15_22/P_pet.tif').flatten()
P_pre  = read_image('/Users/zhangperry/Desktop/11_15_22/P_pre.tif').flatten()
P_tmp  = read_image('/Users/zhangperry/Desktop/11_15_22/P_tmp.tif').flatten()
PHICKL  = read_image('/Users/zhangperry/Desktop/11_15_22/PHICKL.tif').flatten()
PHIHOX  = read_image('/Users/zhangperry/Desktop/11_15_22/PHIHOX.tif').flatten()
Slope  = read_image('/Users/zhangperry/Desktop/11_15_22/slope.tif').flatten()
SLTPPT  = read_image('/Users/zhangperry/Desktop/11_15_22/SLTPPT.tif').flatten()
SNDPPT  = read_image('/Users/zhangperry/Desktop/11_15_22/SNDPPT.tif').flatten()
WWP  = read_image('/Users/zhangperry/Desktop/11_15_22/WWP.tif').flatten()

Feature = {"AK": AK, "Altitude": Altitude, "AN": AN, 
           "AP": AP, "Aspect": Aspect, "Awch": Awch, "AWCts": AWCts, "BDRLOG": BDRLOG, 
           "BDTICM": BDTICM, "BLDFIE": BLDFIE, "CECSOL": CECSOL, "CLYPPT": CLYPPT, "DIF": DIF,
           "LOS": LOS, "OCDENS": OCDENS, "OCSTHA": OCSTHA, "ORCDRC": ORCDRC, "P_pet": P_pet,
           "P_pre": P_pre, "P_tmp": P_tmp, "PHICKL": PHICKL, "PHIHOX": PHIHOX, "Slope":Slope,
           "SLTPPT":SLTPPT, "SNDPPT":SNDPPT, "WWP":WWP}
Feature = pd.DataFrame(Feature)
Feature = Feature.to_numpy()

y_pred_proba = Classifier.predict_proba(Feature)
for i in np.arange(len(Species_list)):
    print(i)
    make_raster('/Users/zhangperry/Desktop/11_15_22/AK.tif',f'{Species_list[i]}_分布概率.tif',y_pred_proba[:,i].reshape(3549, 6199),gdal.GDT_Float32)

In [ ]:
# ---------------使用SMOTE将各类别的样本数达到一致-------------------
X = Forest_data.iloc[:,4:].values
y = Forest_data.iloc[:, 3].values.astype('str')

sm = SMOTE(random_state=42)
X_resample, y_resample = sm.fit_resample(X, y)

# ----------特征筛选的方法有方差筛选、互信息筛选、REFCV筛选------------
# ----------------递归特征消除法，返回特征选择后的数据----------------
# ------------------参数estimator为基模型-------------------------
# -------------参数n_features_to_select为选择的特征个数------------
loc = RFE(estimator=RandomForestClassifier(n_estimators=1300,max_features=1,oob_score=True), n_features_to_select=5).fit(X_resample, y_resample).support_
X_resample = RFE(estimator=RandomForestClassifier(n_estimators=1300,max_features=1,oob_score=True), n_features_to_select=5).fit_transform(X_resample, y_resample)

# --------------------------树种分布建模------------------------------
X_train, X_test, y_train, y_test = train_test_split(X_resample,y_resample,test_size=0.3,random_state=42)
Classifier = RandomForestClassifier(n_estimators=1300,oob_score=True)
Classifier.fit(X_train, y_train)
y_pred = Classifier.predict(X_test)
y_pred_proba = Classifier.predict_proba(X_test)

# ------------预测准确率--------------------
print(metrics.classification_report(y_test, y_pred))
print(metrics.cohen_kappa_score(y_test, y_pred))

# -------------ROC曲线----------------------
Species_list = np.unique(y)
Species_list 
for i in np.arange(len(Species_list)):
    
    ytest = copy.deepcopy(y_test)

    ytest[np.where(ytest != Species_list[i])] = 0
    ytest[np.where(ytest == Species_list[i])] = 1

    y_pred_proba = Classifier.predict_proba(X_test)[:,i]
    fpr, tpr, threshold = metrics.roc_curve(list(np.int32(ytest)), list(y_pred_proba))
    roc_auc = metrics.auc(fpr, tpr)
    print(roc_auc)
    plt.plot(fpr, tpr)
    # plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive rate', size = 18)
plt.xlabel('False Positive rate', size = 18)
plt.show()

# ----------------树种分布概率预测-------------------
# -------------------图像特征------------------------
AK = read_image('/Users/zhangperry/Desktop/11_15_22/AK.tif').flatten()
Altitude  = read_image('/Users/zhangperry/Desktop/11_15_22/altitude.tif').flatten()
AN    = read_image('/Users/zhangperry/Desktop/11_15_22/AN.tif').flatten()
AP   = read_image('/Users/zhangperry/Desktop/11_15_22/AP.tif').flatten()
Aspect   = read_image('/Users/zhangperry/Desktop/11_15_22/aspect.tif').flatten()
Awch   = read_image('/Users/zhangperry/Desktop/11_15_22/awch.tif').flatten()
AWCts   = read_image('/Users/zhangperry/Desktop/11_15_22/AWCts.tif').flatten()
BDRLOG   = read_image('/Users/zhangperry/Desktop/11_15_22/BDRLOG.tif').flatten()
BDTICM   = read_image('/Users/zhangperry/Desktop/11_15_22/BDTICM.tif').flatten()
BLDFIE   = read_image('/Users/zhangperry/Desktop/11_15_22/BLDFIE.tif').flatten()
CECSOL   = read_image('/Users/zhangperry/Desktop/11_15_22/CECSOL.tif').flatten()
CLYPPT   = read_image('/Users/zhangperry/Desktop/11_15_22/CLYPPT.tif').flatten()
DIF  = read_image('/Users/zhangperry/Desktop/11_15_22/DIF.tif').flatten()
LOS  = read_image('/Users/zhangperry/Desktop/11_15_22/LOS.tif').flatten()
OCDENS  = read_image('/Users/zhangperry/Desktop/11_15_22/OCDENS.tif').flatten()
OCSTHA  = read_image('/Users/zhangperry/Desktop/11_15_22/OCSTHA.tif').flatten()
ORCDRC  = read_image('/Users/zhangperry/Desktop/11_15_22/ORCDRC.tif').flatten()
P_pet  = read_image('/Users/zhangperry/Desktop/11_15_22/P_pet.tif').flatten()
P_pre  = read_image('/Users/zhangperry/Desktop/11_15_22/P_pre.tif').flatten()
P_tmp  = read_image('/Users/zhangperry/Desktop/11_15_22/P_tmp.tif').flatten()
PHICKL  = read_image('/Users/zhangperry/Desktop/11_15_22/PHICKL.tif').flatten()
PHIHOX  = read_image('/Users/zhangperry/Desktop/11_15_22/PHIHOX.tif').flatten()
Slope  = read_image('/Users/zhangperry/Desktop/11_15_22/slope.tif').flatten()
SLTPPT  = read_image('/Users/zhangperry/Desktop/11_15_22/SLTPPT.tif').flatten()
SNDPPT  = read_image('/Users/zhangperry/Desktop/11_15_22/SNDPPT.tif').flatten()
WWP  = read_image('/Users/zhangperry/Desktop/11_15_22/WWP.tif').flatten()

Feature = {"AK": AK, "Altitude": Altitude, "AN": AN, 
           "AP": AP, "Aspect": Aspect, "Awch": Awch, "AWCts": AWCts, "BDRLOG": BDRLOG, 
           "BDTICM": BDTICM, "BLDFIE": BLDFIE, "CECSOL": CECSOL, "CLYPPT": CLYPPT, "DIF": DIF,
           "LOS": LOS, "OCDENS": OCDENS, "OCSTHA": OCSTHA, "ORCDRC": ORCDRC, "P_pet": P_pet,
           "P_pre": P_pre, "P_tmp": P_tmp, "PHICKL": PHICKL, "PHIHOX": PHIHOX, "Slope":Slope,
           "SLTPPT":SLTPPT, "SNDPPT":SNDPPT, "WWP":WWP}
Feature = pd.DataFrame(Feature)
Feature = Feature.iloc[:, loc].to_numpy()

y_pred_proba = Classifier.predict_proba(Feature)
for i in np.arange(len(Species_list)):
    print(i)
    make_raster('/Users/zhangperry/Desktop/11_15_22/AK.tif',f'{Species_list[i]}_分布概率_SMOTE.tif',y_pred_proba[:,i].reshape(3549, 6199),gdal.GDT_Float32)